#### ID 2120

```The marketing team is evaluating the performance of their previously ran promotions.They are especially interested in comparing the sales on the first day vs. the last day of each promotion. Segment the results by promotion and find the percentage of transactions that happened on the first and last day of each. Your output should contain the promotion ID, percentage of transactions on the first day, and percentage of transactions on the last day.```

In [ ]:
%%sql
SELECT oo.promotion_id,
       100.0 * AVG(CASE WHEN date = start_date THEN 1 ELSE 0 END) AS pct_first_day,
       100.0 * AVG(CASE WHEN date = end_date THEN 1 ELSE 0 END)   AS pct_first_day
FROM online_sales_promotions AS osp
         JOIN online_orders AS oo ON osp.promotion_id = oo.promotion_id
GROUP BY oo.promotion_id

In [ ]:
df = pd.merge(online_sales_promotions, online_orders, how='inner', on='promotion_id')

# df['is_start'] = df.apply(lambda row: 1 if row['start_date'] == row['date'] else 0, axis=1)
# df['is_end'] = df.apply(lambda row: 1 if row['end_date'] == row['date'] else 0, axis=1)

df['is_start'] = (df['start_date'] == df['date']).astype('int')
df['is_end'] = (df['end_date'] == df['date']).astype('int')

result = df.groupby('promotion_id', as_index=False).agg(
    is_start_date=pd.NamedAgg(column='is_start', aggfunc=lambda series: series.mean() * 100),
    is_end_date=pd.NamedAgg(column='is_end', aggfunc=lambda series: series.mean() * 100)
)

#### ID 2121

```The marketing department is assessing the success of their promotional campaigns. You have been asked to find which products sold the most units for each promotion. Your output should contain the promotion ID, product ID, and corresponding total sales for the most successful product ID. In the case of a tie, output all results.```

In [ ]:
%%sql
WITH cte AS (SELECT promotion_id,
                    product_id,
                    SUM(units_sold)                                                AS total_sales,
                    RANK()
                    OVER (PARTITION BY promotion_id ORDER BY SUM(units_sold) DESC) AS rnk
             FROM online_orders
             GROUP BY promotion_id, product_id)
SELECT promotion_id, product_id, total_sales
FROM cte
WHERE rnk = 1

In [ ]:
df = online_orders

grouped_df = df.groupby(['promotion_id', 'product_id'], as_index=False).agg(total_units=('units_sold', 'sum'))

grouped_df['rnk'] = grouped_df.groupby('promotion_id')['total_units'].rank(method='dense', ascending=False)

grouped_df.query('rnk == 1').drop(columns='rnk')

#### ID 2122

```The VP of Sales feels that some product categories don't sell and can be completely removed from the inventory. As a first pass analysis, they want you to find what percentage of product categories have never been sold.```

In [ ]:
%%sql
SELECT (1 - AVG(sold)) * 100 AS percentage_of_unsold_categories
FROM (SELECT op.product_category,
             CASE WHEN SUM(oo.units_sold) IS NOT NULL THEN 1 ELSE 0 END AS sold
      FROM online_product_categories opc
               LEFT JOIN online_products op ON op.product_category = opc.category_id
               LEFT JOIN online_orders oo USING (product_id)
      GROUP BY op.product_category) t1

#### ID 2124

```You have been tasked with finding the top two single-channel media types (ranked in decreasing order) that correspond to the most money the grocery chain had spent on its promotional campaigns. Your output should contain the media type and the total amount spent on the advertising campaign. In the event of a tie, output all results and do not skip ranks.```

In [ ]:
%%sql
WITH total_cost_by_media_type AS (SELECT media_type,
                                         SUM(cost)                                   AS total_cost,
                                         DENSE_RANK() OVER (ORDER BY SUM(cost) DESC) AS rnk
                                  FROM online_sales_promotions
                                  GROUP BY media_type)
SELECT media_type, total_cost
FROM total_cost_by_media_type
WHERE rnk <= 2

In [ ]:
df = online_sales_promotions

grouped_df = df.groupby('media_type', as_index=False).agg(total_cost=('cost', 'sum'))

grouped_df['rnk'] = grouped_df['total_cost'].rank(method='dense', ascending=False)

grouped_df.query('rnk <= 2').drop(columns='rnk')

#### ID 2127

```Calculate the sales revenue for the year 2021.```

In [ ]:
%%sql
SELECT SUM(order_total) AS revenue
FROM amazon_sales
WHERE DATE_TRUNC('year', order_date) = '2021-01-01'

In [ ]:
df = amazon_sales

df['year'] = df['order_date'].dt.year

df.query('year == 2021')['order_total'].sum()

#### ID 2128

```Calculate the total revenue made per book. Output the book ID and total sales per book. In case there is a book that has never been sold, include it in your output with a value of 0.```

In [ ]:
%%sql
SELECT ab.book_id, COALESCE(SUM(quantity * unit_price), 0) AS total_sales
FROM amazon_books AS ab
         LEFT JOIN amazon_books_order_details AS abod ON ab.book_id = abod.book_id
GROUP BY ab.book_id

In [ ]:
df = pd.merge(amazon_books, amazon_books_order_details, how='left', on='book_id')
df['sales'] = df['quantity'] * df['unit_price']

df.groupby('book_id', as_index=False).agg(total_sales=('sales', 'sum'))

#### ID 2129

```You are given a list of posts of a Facebook user. Find the average number of likes.```

In [ ]:
%%sql
SELECT AVG(no_of_likes) AS avg_likes
FROM fb_posts

In [ ]:
df = fb_posts

df['no_of_likes'].mean()